In [38]:
from datetime import datetime
import pandas as pd 
from os import rename, sep
from textwrap import indent
import numpy as np 
import io
from datetime import timedelta, date, datetime
from os import listdir
import plotly.express as px
from os.path import isfile, join
dt_string = datetime.today().strftime('%d%m%y')
import plotly.graph_objects as go
import dash
from dash import html ,dcc

In [12]:
with open('config\path.txt', "r") as tf:
        path = tf.read()
        path = path.replace("\\","/")
tf.close()
alertas_x_hora = pd.DataFrame()
for i in listdir(path):
        files_store= pd.DataFrame()
        if i.startswith('JPPs - Análisis de notas crédito - '):
                new_path= f"{path}/{i}"
                files_names = [f for f in listdir(new_path) if isfile(join(new_path, f))]
                for i in files_names:
                        if  i.endswith('.xlsx'):
                                file = pd.read_excel(f'{new_path}/{i}',sheet_name="Alertas x hora",dtype=str)
                                files_store = pd.concat([files_store,file])
        alertas_x_hora = pd.concat([alertas_x_hora, files_store])   

In [13]:
alertas_x_hora.loc[(alertas_x_hora["Grabación?"].notna()) & (alertas_x_hora["Cliente?"].notna()) & (alertas_x_hora["Producto?"].notna()), "indicador"] = "Diligenciado"
alertas_x_hora.loc[alertas_x_hora["indicador"] != "Diligenciado", "indicador"] = "No diligenciado"

In [14]:
h_x_fig = alertas_x_hora.groupby(["Desc_local","indicador"]).agg({"Cautoriza":"nunique",}).reset_index().sort_values("Cautoriza", ascending=False)

In [17]:
h_x_fig

,Desc_local,indicador,Cautoriza
25,TITAN,No diligenciado,249
8,COLINA,Diligenciado,231
15,JARDIN PLAZA,No diligenciado,198
13,GALERIAS,No diligenciado,174
26,UNICENTRO,No diligenciado,160
7,CENTRO MAYOR,No diligenciado,159
23,SANTAFE MEDELLIN,No diligenciado,158
22,SANTA FE,No diligenciado,153
4,CACIQUE,No diligenciado,137
12,FONTANAR,No diligenciado,128


In [34]:
dil = h_x_fig.loc[h_x_fig.indicador == "diligenciado" ]
no_dil = h_x_fig.loc[h_x_fig.indicador == "No diligenciado" ]

In [37]:
trace1 = go.Bar(x=dil["Desc_local"], y=dil["indicador"], name="Diligenciado")
trace2 = go.Bar(x=no_dil["Desc_local"], y=no_dil["indicador"], name="No Diligenciado")

In [44]:
app = dash.Dash()
app.layout = html.Div(children=[
    html.H1(children="Alarmas por hora"),
    html.Div(children="Alarmas por hora"),
    dcc.Graph(
        id = "ejemplo",
        figure = {
            "data": [trace1,trace2], 
            "layout": go.Layout(title ="Alertas por hora",barmode="stack")
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

In [15]:
h_fig = px.bar(h_x_fig, x="Desc_local", y="Cautoriza",color="indicador",text="Cautoriza",color_discrete_sequence=['rgb(170, 57, 57)','rgb(45, 136, 45)'], title="Alertas por hora", labels={"Desc_local":"Local","Cautoriza":"Cantidad NCs"}) 
h_fig.update_layout(xaxis_categoryorder = 'total descending',legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.7))
h_fig.show()
# h_fig.write_image(f"output/{dt_string}_alertas_x_hora.svg",height =500,width=900 )

In [ ]:
with open('config\path.txt', "r") as tf:
        path = tf.read()
        path = path.replace("\\","/")
tf.close()
alertas_x_monto = pd.DataFrame()
for i in listdir(path):
        files_store= pd.DataFrame()
        if i.startswith('JPPs - Análisis de notas crédito - '):
                new_path= f"{path}/{i}"
                files_names = [f for f in listdir(new_path) if isfile(join(new_path, f))]
                for i in files_names:
                        if  i.endswith('.xlsx'):
                                file = pd.read_excel(f'{new_path}/{i}',sheet_name="Alertas x monto",dtype=str)
                                files_store = pd.concat([files_store,file])
        alertas_x_monto = pd.concat([alertas_x_monto, files_store])

In [ ]:
alertas_x_monto.loc[alertas_x_monto["cliente?"].notna(),"Cliente?"] = alertas_x_monto.loc[alertas_x_monto["cliente?"].notna(), "cliente?"].values
alertas_x_monto.loc[alertas_x_monto["Producto"].notna(),"Producto?"] = alertas_x_monto.loc[alertas_x_monto["Producto"].notna(), "Producto"].values
alertas_x_monto.loc[alertas_x_monto.C.notna(),"Cautoriza" ] = alertas_x_monto.loc[alertas_x_monto.C.notna(),"C"].values 
alertas_x_monto.loc[alertas_x_monto.CC.notna(),"Cautoriza" ] = alertas_x_monto.loc[alertas_x_monto.CC.notna(),"CC"].values 
alertas_x_monto.loc[alertas_x_monto["S"].notna(), "Cautoriza"]  = alertas_x_monto.loc[alertas_x_monto["S"].notna(), "S" ].values


In [ ]:
alertas_x_monto.drop(columns=['Unnamed: 16', 'C', 'CC', 'Producto', 'cliente?', 'S',
       'Unnamed: 17'],inplace=True)

In [ ]:
alertas_x_monto.loc[(alertas_x_monto["Grabación?"].notna()) & (alertas_x_monto["Cliente?"].notna()) & (alertas_x_monto["Producto?"].notna()), "indicador"] = "Diligenciado"
alertas_x_monto.loc[alertas_x_monto["indicador"] != "Diligenciado", "indicador"] = "No diligenciado"

In [ ]:
m_x_fig = alertas_x_monto.groupby(["Desc_local","indicador"]).agg({"Cautoriza":"nunique",}).reset_index().sort_values("Cautoriza", ascending=False)

In [ ]:
h_fig = px.bar(m_x_fig, x="Desc_local", y="Cautoriza",color="indicador",text="Cautoriza",color_discrete_sequence=['rgb(170, 57, 57)','rgb(45, 136, 45)'], title="Alertas por monto", labels={"Desc_local":"Local","Cautoriza":"Cantidad NCs"}) 
h_fig.update_layout(xaxis_categoryorder = 'total descending',legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.7))
h_fig.show()
h_fig.write_image(f"output/{dt_string}_alertas_x_monto.svg",height =500,width=900 )

In [ ]:
alertas_x_monto

,Cautoriza,Desc_local,Dcompra_nvo,Nterminal_nvo,Hora,Cod_Empleado,Num_Documento,nombre_completo,Cargo,SKU,...,Producto?,Nsecuencia_nvo,Unnamed: 16,C,CC,Producto,cliente?,S,Unnamed: 17,indicador
0,1070783916,ACQUA,2021-11-10 00:00:00,36,1114,155586,65778038,DIANA JIMENA TORRES,VENDEDOR INTEGRAL,1,...,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado
1,1070785859,ACQUA,2021-11-10 00:00:00,96,1140,162507,1110487738,DIANA MARCELA RODRIGUEZ,ASESOR MULTISERVICIOS,1,...,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado
2,1070788294,ACQUA,2021-11-10 00:00:00,96,1205,107174,1110578449,GLADYS MERCEDES RUBIANO,VENDEDOR INTEGRAL,1,...,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado
3,1070788911,ACQUA,2021-11-10 00:00:00,96,1213,162507,1110487738,DIANA MARCELA RODRIGUEZ,ASESOR MULTISERVICIOS,1,...,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado
4,1070789574,ACQUA,2021-11-10 00:00:00,36,1223,155586,65778038,DIANA JIMENA TORRES,VENDEDOR INTEGRAL,1,...,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diligenciado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,1075794122,WTC CALI,2022-02-20 00:00:00,96,1536,105783,1144027728,MARTHA LILIANA IMBAJOA,ASESOR MULTISERVICIOS,1,...,NaN,1710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No diligenciado
16,1075796676,WTC CALI,2022-02-20 00:00:00,96,1623,182273,1109002800,JHONATAN ANDRES SANCHEZ,VENDEDOR INTEGRAL,1,...,NaN,1712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No diligenciado
17,1075801134,WTC CALI,2022-02-20 00:00:00,10,1744,200735,1111755212,KELLY LEANDRA CHUNGA,OPERADOR DE P.O.S.,1,...,NaN,2446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No diligenciado
18,1075802124,WTC CALI,2022-02-20 00:00:00,30,1818,132411,1130638803,JHON EDINSON CASTAÑO,VENDEDOR INTEGRAL,2,...,NaN,861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No diligenciado
